In [44]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import numpy.random as rn
import networkx as nx
import my_networkx as my_nx

G = nx.DiGraph() 

In [46]:
A = np.array([[0, 1, 0, 0, 0, 0],[1, 0, 1, 0, 1, 0],[0, 1, 0, 1, 0, 0],[0, 0, 1, 0, 1, 0],[0, 1, 0, 1, 0, 1],[0, 0, 0, 0, 1, 0]])
print(A)

[[0 1 0 0 0 0]
 [1 0 1 0 1 0]
 [0 1 0 1 0 0]
 [0 0 1 0 1 0]
 [0 1 0 1 0 1]
 [0 0 0 0 1 0]]


In [70]:
def plot_network(A, w=np.ones_like(A), position={1:(0,0),2:(1,0),3:(2,1),4:(3,0),5:(2,-1),6:(2,0)}, name="figures/Example_Calmodulin_1.pdf"):
    edges = []

    for i in range(6): 
        for j in range(6): 
            if A[i, j] == 1: 
                #G.add_edge(i,j)
                edges.append((i+1, j+1, weight=w[i,j], {"w":r"$r_{%d%d}$" % (i+1, j+1)}))
        G.add_node(i+1)

    G.add_edges_from(edges)


    fixed_positions = {1:(0,0),2:(1,0),3:(2,1),4:(3,0),5:(2,-1),6:(2,0)}#dict with two of the positions set
    fixed_nodes = fixed_positions.keys()
    pos = nx.spring_layout(G, pos=fixed_positions, fixed=fixed_nodes)

    node_sizes = [3 + 10 * i for i in range(len(G))]
    M = G.number_of_edges()
    edge_colors = range(2, M + 2)
    edge_alphas = [(5 + i) / (M + 4) for i in range(M)]
    cmap = plt.cm.plasma

    fig, ax = plt.subplots(figsize=(15, 10))
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=3000, edgecolors="k")
    nx.draw_networkx_labels(G, pos, ax=ax, font_size=30)

    curved_edges = [edge for edge in G.edges() if reversed(edge) in G.edges()]
    straight_edges = list(set(G.edges()) - set(curved_edges))
    nx.draw_networkx_edges(G, pos, ax=ax, edgelist=straight_edges, arrowstyle="<|-", width=5, node_size=3000)
    arc_rad = 0.25
    nx.draw_networkx_edges(G, pos, ax=ax, edgelist=curved_edges, connectionstyle=f'arc3, rad = {arc_rad}', arrowstyle="<|-", width=2, node_size=3000)

    edge_weights = nx.get_edge_attributes(G,'w')
    curved_edge_labels = {edge: edge_weights[edge] for edge in curved_edges}
    straight_edge_labels = {edge: edge_weights[edge] for edge in straight_edges}
    my_nx.my_draw_networkx_edge_labels(G, pos, ax=ax, edge_labels=curved_edge_labels,rotate=False,rad = arc_rad, font_size=20)
    nx.draw_networkx_edge_labels(G, pos, ax=ax, edge_labels=straight_edge_labels,rotate=False)
    plt.savefig(name, dpi=500, bbox_inches="tight")

plot_network(A)

SyntaxError: invalid syntax (<ipython-input-70-de5991e9d539>, line 8)

In [3]:
energies = rn.exponential(1, size=(6))
print(energies)

[1.08043414 1.77675376 2.33751699 1.05233855 0.853644   0.91091134]


In [4]:
peq = np.exp(-energies)
Z = np.sum(peq)
peq = peq/Z

In [5]:
N = len(peq)
Pmatrix = np.zeros((N, N))
for i in range(N):
    Pmatrix[i, i] = np.sqrt(peq[i])

print(Pmatrix)

[[0.43640673 0.         0.         0.         0.         0.        ]
 [0.         0.30809706 0.         0.         0.         0.        ]
 [0.         0.         0.23276591 0.         0.         0.        ]
 [0.         0.         0.         0.44258055 0.         0.        ]
 [0.         0.         0.         0.         0.48880799 0.        ]
 [0.         0.         0.         0.         0.         0.47501011]]


In [6]:
L = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        if A[i, j] == 1:
            L[i, j] = np.exp(energies[j])
    L[i, i] = - np.sum(L[i, :])

Ls = np.matmul(np.linalg.inv(Pmatrix), np.matmul(L, Pmatrix))
print(Ls)

ew, ev = np.linalg.eigh(Ls)
print(ew)

D = np.matmul(ev.T, np.matmul(Ls, ev))
print(D)



[[ -5.91063786   4.17282785   0.           0.           0.
    0.        ]
 [  4.17282785 -15.64963812   7.82352619   0.           3.72549182
    0.        ]
 [  0.           7.82352619  -8.77497955   5.44625256   0.
    0.        ]
 [  0.           0.           5.44625256 -12.7036799    2.5934558
    0.        ]
 [  0.           3.72549182   0.           2.5934558  -11.26156718
    2.4163972 ]
 [  0.           0.           0.           0.           2.4163972
   -2.34818807]]
[-2.35646144e+01 -1.41502738e+01 -1.13673361e+01 -5.60752857e+00
 -1.95893785e+00 -2.80409745e-15]
[[-2.35646144e+01 -1.07552856e-15 -7.85482790e-15  2.06952511e-15
  -7.07767178e-16 -1.15661824e-15]
 [ 7.00828284e-16 -1.41502738e+01  4.45476989e-15  1.24379673e-15
  -1.21430643e-15  3.34422134e-15]
 [-7.79931675e-15  4.57966998e-15 -1.13673361e+01 -1.38777878e-16
  -7.21644966e-16 -6.85071384e-16]
 [ 2.55351296e-15  1.10675358e-15 -2.77555756e-16 -5.60752857e+00
  -4.57966998e-16  2.03905248e-15]
 [ 4.44089210e-1